In [1]:
import pandas as pd
import numpy as np
import re
import funciones as f

In [2]:
df = pd.read_csv("./input/GSAF5.csv", encoding="ISO-8859-1")
df.head(2)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN


In [3]:
df_clean = f.clean_colum(df)
df.columns


Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number_1', 'case_number_2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

In [4]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

drop_cols = list(null_cols[null_cols > 4000].index)
df_clean = df.drop(drop_cols, axis=1)
df_clean.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex,...,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992


In [5]:
df_clean = df.drop(axis = 1,columns = ['unnamed:_22', 'unnamed:_23', 'href', 'href_formula', 'pdf', 'original_order',
                                  'investigator_or_source', 'injury', 'case_number_1', 
                                  'case_number_2', 'type', 'name', 'case_number', 'age', 'case_number'])
df_clean.head(2)

,date,year,country,area,location,activity,sex,fatal_(y/n),time,species
0,18-Sep-16,2016,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N,13h00,NaN
1,18-Sep-16,2016,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,N,11h00,NaN


In [6]:
null = df_clean.isnull().sum()
null

date              0
year              0
country          43
area            402
location        496
activity        527
sex             567
fatal_(y/n)      19
time           3213
species        2934
dtype: int64

In [7]:
df_hyp = df_clean[(df_clean["country"]=="AUSTRALIA") | (df_clean["country"]=="USA")][["activity", "country", "fatal_(y/n)", "year"]]

In [8]:
df_hyp["activity"].replace()


0            Surfing
1            Surfing
2            Surfing
3            Surfing
4            Surfing
            ...     
5985         Fishing
5986        Swimming
5987          Diving
5988    Pearl diving
5989        Swimming
Name: activity, Length: 3395, dtype: object

In [9]:
#TEST FUNCTION
a = set(df_hyp["activity"].value_counts().index)
for ele in a: 
    print(ele," -------" ,f.clean_surf(ele))

Wading  ------- Wading
Freediving for seashells  ------- Freediving for seashells
Body surfing / treading water  ------- Body surfing / treading water
Playing in the surf with his 2 dogs  ------- Playing in the surf with his 2 dogs
Swimming from camp  ------- Swimming from camp
Sitting in a boat  ------- Sitting in a boat
Fishing, lifting shark out of craypot  ------- Fishing, lifting shark out of craypot
Testing anti-shark cage  ------- Testing anti-shark cage
Paddling on surfboard  ------- Paddling on surfboard
Skindiving for abalone (but at surface)  ------- Skindiving for abalone (but at surface)
Trailing the field in a surf race  ------- Trailing the field in a surf race
Fishing for albacore  ------- Fishing for albacore
Kite Boarding  ------- Kite Boarding
Wreck of the Sovereign  ------- Wreck of the Sovereign
Surf skiing  ------- surfing
Harpooning turtles  ------- Harpooning turtles
After rowing skiff was holed by shark, he was attempting to swim ashore  ------- After rowing sk

In [10]:
#df_hyp["activity"].value_counts()
df_hyp["activity"] = df_hyp["activity"].apply(f.clean_surf)

In [11]:
#Function Test
a = set(df_hyp["fatal_(y/n)"].value_counts().index)
for ele in a:
    print(ele, "-----", f.clean_fatal(ele))

N  ----- False
UNKNOWN ----- False
Y ----- True
 N ----- False
N ----- False
n ----- False


In [16]:
df_hyp["fatal_(y/n)"] = df_hyp["fatal_(y/n)"].apply(f.clean_fatal)
df_hyp=df_hyp.rename(columns = {'fatal_(y/n)':'is_fatal'})
df_hyp.value_counts()

KeyError: 'fatal_(y/n)'

In [13]:
df_hyp[(df_hyp["activity"] == "surfing") & (df_hyp["country"] == "USA") & (df_hyp["is_fatal"] == True)]["is_fatal"].count()

13

In [14]:
df_hyp[(df_hyp["activity"] == "surfing") & (df_hyp["country"] == "AUSTRALIA") & (df_hyp["is_fatal"] == True)]["is_fatal"].count()

14

In [15]:
df_hyp[(df_hyp["activity"] == "surfing") & (df_hyp["country"] == "USA")]["is_fatal"]

0       False
1       False
2       False
13      False
14      False
        ...  
4058    False
4140    False
5808     True
5837     True
5891    False
Name: is_fatal, Length: 577, dtype: bool